In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, AveragePooling2D, Input
from keras.optimizers import Adam
from statistics import mean
import tensorflow as tf
from keras.models import model_from_json
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                    #rotation_range=30,
                                    #shear_range=0.3,
                                    #zoom_range=0.3,
                                    #horizontal_flip=True,
                                    #fill_mode='nearest'
                                   )
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                    #rotation_range=30,
                                    #shear_range=0.3,
                                    #zoom_range=0.3,
                                    #horizontal_flip=True,
                                    #fill_mode='nearest'
                                    )

train_generator = train_data_generator.flow_from_directory(
    'data/train',
    target_size=(48, 48),
    batch_size=32,
    color_mode="grayscale",
    class_mode='categorical')

validation_generator = test_data_generator.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical')

In [ ]:
#Initialize our facial emotion recognition model
FER_CNN = Sequential()

#Add the input dimensions
FER_CNN.add(Input(shape=(48,48,1)))

#Add two convolutional layers with kernel size 5, and relu activation
FER_CNN.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
FER_CNN.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
#Add a max pooling layer to reduce noise, with kernel 2
FER_CNN.add(MaxPooling2D(pool_size=(2, 2)))
#Add a dropout layer with rate 0.25 to reduce risk of overfitting
FER_CNN.add(Dropout(0.25))
 
#Two convolutional layers with kernel size 5 and activation relu
#followed by two maxpooling layers with kernel 2 (to reduce noise)
FER_CNN.add(Conv2D(128, kernel_size=(5, 5), activation='relu'))
FER_CNN.add(MaxPooling2D(pool_size=(2, 2)))
FER_CNN.add(Conv2D(128, kernel_size=(5, 5), activation='relu'))
FER_CNN.add(MaxPooling2D(pool_size=(2, 2)))
#Dropout layer to reduce risk of overfitting, rate of 0.25
FER_CNN.add(Dropout(0.25))
 
#Final linear layers with final dropout layer with rate 0.25
FER_CNN.add(Flatten())
#FER_CNN.add(Dense(1024, activation='relu'))
FER_CNN.add(Dropout(0.25))
#Softmax function to give probabilistic distribution to each class
FER_CNN.add(Dense(7, activation='softmax'))
 
#See our model architecture:
FER_CNN.summary()
 
 
learning_rate = 0.001

#Choose our optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
FER_CNN.compile(loss='categorical_crossentropy', optimizer=optimizer, 
                      metrics=['accuracy'])

In [ ]:
emotion_model_info = FER_CNN.fit(
    train_generator,
    steps_per_epoch=28709 // 32,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 32
)

In [ ]:
FER_CNN.evaluate(validation_generator)
train_accuracy = emotion_model_info.history['accuracy']
validation_accuracy = emotion_model_info.history['val_accuracy']
loss = emotion_model_info.history['loss']

max_val_accuracy = max(validation_accuracy)
print("Test Accuracy: ", max_val_accuracy)
print("Training Accuracy: ", max(train_accuracy))

overall_accuracies = []
for i in range(len(train_accuracy)):
    overall_accuracies.append((train_accuracy[i]*28709 + validation_accuracy[i]*7178) / (28709+7178))
print("Overall Accuracy: ", max(overall_accuracies))

In [ ]:
# Accuracy graph
plt.subplot(1, 2, 1)
plt.plot(train_accuracy, label='Training Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
 
# loss graph
plt.subplot(1, 2, 2)
plt.plot(loss, label='Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
 
plt.show()